In [3]:
import openai
import os


openai.api_base = os.environ["OPENAI_API_HOST"]
openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_type = os.environ["OPENAI_API_TYPE"]
openai.api_version = os.environ["OPENAI_API_VERSION"]


In [13]:
def check_code(code):
    result = openai.ChatCompletion.create(
        engine = "gpt-35-turbo",
        messages = [
            {"role": "system", "content": "You are security checker. You analyse the source code and check for security vulnerabilities. You are quested to check the following code snippet. The concern I have is if this code could potenitally send some information to a remote server. If you see that the code might do that, respond the single word 'BAD', otherwise respond the single word 'OK'. Do not add any comments. Acknowledge."},
            {"role": "assistant", "content": "Acknowledge."},
            {"role": "user", "content": code}
        ]
    )
    return result.choices[0].message.content


In [83]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def count_tokens_in_file(file):
    #print(file)
    with open(file, 'r') as f:
        code = f.read()
        return len(encoding.encode(code))
    
def find_all_files_except_git_folder(path):
    result = []
    for root, dirs, files in os.walk(path):
        for file in files:
            newfile = os.path.join(root, file)
            if not root.startswith('./.git') and not root.startswith('./venv/') \
                and not file.endswith('.png') and not file.endswith('.ico') \
                    and not file.endswith('.jpg') and not file.endswith('.gif') \
                        and not file == "package-lock.json":
                result.append((newfile, count_tokens_in_file(newfile)))
            else:
                print(f"ignoring {newfile}")
    return result

In [84]:
files_data = find_all_files_except_git_folder(".")

ignoring ./package-lock.json
ignoring ./public/favicon.ico
ignoring ./public/screenshot.png
ignoring ./public/screenshots/screenshot-0402023.jpg
ignoring ./.github/workflows/deploy-docker-image.yaml
ignoring ./.github/workflows/run-test-suite.yml
ignoring ./venv/bin/jupyter-run
ignoring ./venv/bin/wheel3.10
ignoring ./venv/bin/wheel3
ignoring ./venv/bin/activate.ps1
ignoring ./venv/bin/python3
ignoring ./venv/bin/python
ignoring ./venv/bin/pip3
ignoring ./venv/bin/ipython
ignoring ./venv/bin/activate.fish
ignoring ./venv/bin/pip-3.10
ignoring ./venv/bin/wheel
ignoring ./venv/bin/wheel-3.10
ignoring ./venv/bin/python3.10
ignoring ./venv/bin/activate_this.py
ignoring ./venv/bin/ipython3
ignoring ./venv/bin/pip
ignoring ./venv/bin/tqdm
ignoring ./venv/bin/jupyter-troubleshoot
ignoring ./venv/bin/pygmentize
ignoring ./venv/bin/jupyter-migrate
ignoring ./venv/bin/pip3.10
ignoring ./venv/bin/activate
ignoring ./venv/bin/activate.nu
ignoring ./venv/bin/normalizer
ignoring ./venv/bin/jupyter-k

In [64]:
files_data.sort(key=lambda x: x[1], reverse=True)

In [77]:
results_bad = []
results_other = []
for f, t_num in files_data:
    print(f"DEBUG: checking {f} with {t_num} tokens")
    with open(f, 'r') as ff:
        code = ff.read()
    res = check_code(code)
    if res == 'BAD':
        results_bad.append(f)
    else:
        results_other.append((f, res))

DEBUG: checking ./components/Chat/Chat.tsx with 3429 tokens
DEBUG: checking ./components/Chat/ChatInput.tsx with 3040 tokens
DEBUG: checking ./pages/api/home/home.tsx with 2654 tokens
DEBUG: checking ./components/Chat/ChatMessage.tsx with 2163 tokens
DEBUG: checking ./public/locales/te/chat.json with 2119 tokens
DEBUG: checking ./public/locales/si/chat.json with 1912 tokens
DEBUG: checking ./testme.ipynb with 1780 tokens
DEBUG: checking ./components/Chatbar/components/PluginKeys.tsx with 1649 tokens
DEBUG: checking ./public/locales/bn/chat.json with 1623 tokens
DEBUG: checking ./components/Chatbar/Chatbar.tsx with 1606 tokens
DEBUG: checking ./__tests__/utils/app/importExports.test.ts with 1558 tokens
DEBUG: checking ./components/Chat/SystemPrompt.tsx with 1496 tokens
DEBUG: checking ./components/Chatbar/components/Conversation.tsx with 1141 tokens
DEBUG: checking ./components/Folder/Folder.tsx with 1135 tokens
DEBUG: checking ./components/Promptbar/components/PromptModal.tsx with 1098

In [82]:
import pprint

pprint.pprint(results_other, indent=4)

[   ('./components/Chat/ChatInput.tsx', 'OK.'),
    ('./pages/api/home/home.tsx', 'OK'),
    ('./components/Chat/ChatMessage.tsx', 'OK'),
    ('./public/locales/te/chat.json', 'OK.'),
    ('./public/locales/si/chat.json', 'OK.'),
    ('./testme.ipynb', 'OK.'),
    ('./components/Chatbar/components/PluginKeys.tsx', 'OK'),
    ('./public/locales/bn/chat.json', 'OK.'),
    ('./components/Chatbar/Chatbar.tsx', 'OK'),
    ('./__tests__/utils/app/importExports.test.ts', 'OK'),
    ('./components/Chat/SystemPrompt.tsx', 'OK.'),
    ('./components/Chatbar/components/Conversation.tsx', 'OK'),
    ('./components/Folder/Folder.tsx', 'OK'),
    ('./components/Promptbar/components/PromptModal.tsx', 'OK'),
    ('./pages/api/google.ts', 'OK'),
    ('./utils/app/importExport.ts', 'OK.'),
    ('./public/locales/ar/chat.json', 'OK.'),
    ('./components/Promptbar/Promptbar.tsx', 'OK.'),
    ('./public/locales/he/chat.json', 'OK'),
    ('./README.md', 'OK.'),
    ('./components/Chat/VariableModal.tsx', '

In [85]:
results_bad

['./components/Chat/Chat.tsx']

In [87]:
def check_code_details(code):
    result = openai.ChatCompletion.create(
        engine = "gpt-35-turbo",
        messages = [
            {"role": "system", "content": "You are security checker. You analyse the source code and check for security vulnerabilities. You are quested to check the following code snippet. The concern I have is if this code could potenitally send some information to a remote server. If you see that the code might do that, respond with information on which lines look suspicious. Tell line numbers explicitly. Acknowledge."},
            {"role": "assistant", "content": "Acknowledge."},
            {"role": "user", "content": code}
        ]
    )
    return result.choices[0].message.content

with open("./components/Chat/Chat.tsx", "r") as f:
    code = f.read()
    print(check_code_details(code))

There is a possibility that this code sends some information to a remote server. Specifically, the code sends a POST request to an API endpoint on line 64, passing a JSON payload as the request body which contains potentially sensitive information such as the chat messages and API key. The payload is also stringified to make it more readable.

Additionally, on line 59, the code fetches a plugin using the `getEndpoint` function which could potentially send requests to a remote server if it is a web API.

It is important to note that the code could be sending information to a remote server without malicious intent, as it may be necessary for chat functionality. However, it is important to analyze and ensure that the information being sent is secure and handled properly to avoid potential data breach issues.
